# Uploading the Models an Creating Twins
This model is intentionally tiny to display the simulation and change capabilities. Unlike the last example, we'll update all of the models and twins in one notebook. Then we can go to the simulation part. 



![Alt text](/docs/img/pipes_n_tanks.png?raw=true "pipes and tanks")

## Connecting

In [42]:
from azure.identity import AzureCliCredential
from azure.digitaltwins.core import DigitalTwinsClient

# using yaml instead of 
import yaml
import uuid

# using altair instead of matplotlib for vizuals
import altair as alt
import numpy as np
import pandas as pd




params = yaml.safe_load(open('../../config.yml'))

azure_cli = AzureCliCredential()
service_client = DigitalTwinsClient(
    params["ADT_HOST_NAME"], azure_cli)
service_client

Loading the model from a local Json file, I'm defining a model for a `Patron`. That's like a `class` that will be used to make several instances of a customer.

## Uploading models

In [43]:
models = [yaml.safe_load(open(os.path.join('models',m))) for m in os.listdir('models')]
models


[{'@id': 'dtmi:com:adt:dtsample:equipment;1',
  '@type': 'Interface',
  '@context': 'dtmi:dtdl:context;2',
  'displayName': 'equipment',
  'contents': [{'@type': 'Property',
    'name': 'model_no',
    'displayName': 'Model No.',
    'schema': 'double',
    'comment': 'model number of the equempent'},
   {'@type': 'Property', 'name': 'name', 'schema': 'string'},
   {'@type': 'Property',
    'name': 'manufacturer',
    'displayName': 'Manufacturer',
    'schema': 'string',
    'comment': 'name of the vendor or manufacturer'}]},
 {'@id': 'dtmi:billmanh:pipe;1',
  '@type': 'Interface',
  'displayName': 'Pipe',
  'extends': 'dtmi:com:adt:dtsample:equipment;1',
  'contents': [{'@type': 'Property',
    'name': 'diameter',
    'displayName': 'diameter (cm)',
    'schema': 'double',
    'comment': 'diameter at center of the tank in centimeters'},
   {'@type': 'Property',
    'name': 'valve',
    'displayName': 'valve (pct)',
    'schema': 'double',
    'comment': 'percent of the valve that is 

The client takes a list of models. So you can just upload all of your models in one go. 

Note that it will crash if the model already exists. 

In [44]:
service_client.create_models(models)

## Uploading the twins

To avoid having to filter out `nan` values I'm using -999. That way I can omit those fields from the json if that twin doesn't have said propperty. 

In [53]:
# Get the Patron model
twin_df = pd.read_excel('equipment.xlsx').fillna(-999)
twin_df

,class,model_id,name,manufacturer,model_no,diameter,valve,capacity,volume,lat,long,elevation
0,tank,dtmi:billmanh:tank;1,large_tank,ACME,1234,500,-999.0,-999.0,500.0,0,0,0
1,tank,dtmi:billmanh:tank;1,small_tank,ACME,1234,100,-999.0,-999.0,200.0,0,0,0
2,pipe,dtmi:billmanh:pipe;1,pipe1,ACME,2345,10,0.0,10.0,-999.0,0,0,0
3,pipe,dtmi:billmanh:pipe;1,pipe2,ACME,2345,10,0.0,10.0,-999.0,0,0,0
4,pipe,dtmi:billmanh:pipe;1,pipe3,ACME,2345,10,0.0,10.0,-999.0,0,0,0


In [54]:
def create_twin_json(x):
    digital_twin_id = x['class'] + str(uuid.uuid4())

    j = {
    "$metadata": {
        "$model": x['model_id']
        }
    }
    for i in [k for k in list(x.keys()) if (k not in ['class','model_id']) & (x[k]!=-999)]:
        j[i]=x[i]
    return digital_twin_id,j
        

In [55]:
twins = [create_twin_json(i) for i in list(twin_df.T.to_dict().values())]
twins[0]

('tank74c0258f-2af8-4a62-95ba-f7a9803e47bd',
 {'$metadata': {'$model': 'dtmi:billmanh:tank;1'},
  'name': 'large_tank',
  'manufacturer': 'ACME',
  'model_no': 1234,
  'diameter': 500,
  'volume': 500.0,
  'lat': 0,
  'long': 0,
  'elevation': 0})

Now I can upload all of them. Note that my process makes a random GUI so you need to query in order to get them back. 

in order to make the DTID a little more readable, I'm adding `name`+`guid`

In [57]:
for i in twins:
    print(f"uploading: {i[0]}; {i[1]['name']}")
    service_client.upsert_digital_twin(i[0],i[1])

uploading: tank74c0258f-2af8-4a62-95ba-f7a9803e47bd; large_tank
uploading: tankdcd691eb-b335-420d-87f3-d5c3bb3fe2d4; small_tank
uploading: pipe0791abba-7fc1-49b1-ba25-126d133db4af; pipe1
uploading: piped83024e9-776c-444e-8f25-04d83c7b227c; pipe2
uploading: pipe0e49be47-fef2-45a9-b9f7-a0364ec9365d; pipe3


## Building the relationships
In this simulation: 

* `large_tank` -> `connectsTo` -> `pipe1`
* `pipe1` -> `connectsTo` -> `pipe2`
* `pipe2` -> `connectsTo` -> `pipe3`
* `pipe2` -> `connectsTo` -> `small_tank`


In [58]:
def make_connection(source,target):
    c = {
        "$relationshipId": f"{source[0]}connectsTo{target[0]}",
        "$sourceId": source[0],
        "$relationshipName": "connectsTo",
        "$targetId": target[0],
        "is_open": True
    }
    return c

To make it easy, I'm just going to use the dataset that they came in.

In [59]:
twin_df

,class,model_id,name,manufacturer,model_no,diameter,valve,capacity,volume,lat,long,elevation
0,tank,dtmi:billmanh:tank;1,large_tank,ACME,1234,500,-999.0,-999.0,500.0,0,0,0
1,tank,dtmi:billmanh:tank;1,small_tank,ACME,1234,100,-999.0,-999.0,200.0,0,0,0
2,pipe,dtmi:billmanh:pipe;1,pipe1,ACME,2345,10,0.0,10.0,-999.0,0,0,0
3,pipe,dtmi:billmanh:pipe;1,pipe2,ACME,2345,10,0.0,10.0,-999.0,0,0,0
4,pipe,dtmi:billmanh:pipe;1,pipe3,ACME,2345,10,0.0,10.0,-999.0,0,0,0


In [60]:
relationships = []

# * `large_tank` -> `connectsTo` -> `pipe1`
relationships.append(make_connection(twins[0],twins[2]))
# * `pipe1` -> `connectsTo` -> `pipe2`
relationships.append(make_connection(twins[2],twins[3]))
# * `pipe2` -> `connectsTo` -> `pipe3`
relationships.append(make_connection(twins[3],twins[4]))
# * `pipe3` -> `connectsTo` -> `small_tank`
relationships.append(make_connection(twins[4],twins[1]))

In [61]:
relationships

[{'$relationshipId': 'tank74c0258f-2af8-4a62-95ba-f7a9803e47bdconnectsTopipe0791abba-7fc1-49b1-ba25-126d133db4af',
  '$sourceId': 'tank74c0258f-2af8-4a62-95ba-f7a9803e47bd',
  '$relationshipName': 'connectsTo',
  '$targetId': 'pipe0791abba-7fc1-49b1-ba25-126d133db4af',
  'is_open': True},
 {'$relationshipId': 'pipe0791abba-7fc1-49b1-ba25-126d133db4afconnectsTopiped83024e9-776c-444e-8f25-04d83c7b227c',
  '$sourceId': 'pipe0791abba-7fc1-49b1-ba25-126d133db4af',
  '$relationshipName': 'connectsTo',
  '$targetId': 'piped83024e9-776c-444e-8f25-04d83c7b227c',
  'is_open': True},
 {'$relationshipId': 'piped83024e9-776c-444e-8f25-04d83c7b227cconnectsTopipe0e49be47-fef2-45a9-b9f7-a0364ec9365d',
  '$sourceId': 'piped83024e9-776c-444e-8f25-04d83c7b227c',
  '$relationshipName': 'connectsTo',
  '$targetId': 'pipe0e49be47-fef2-45a9-b9f7-a0364ec9365d',
  'is_open': True},
 {'$relationshipId': 'pipe0e49be47-fef2-45a9-b9f7-a0364ec9365dconnectsTotankdcd691eb-b335-420d-87f3-d5c3bb3fe2d4',
  '$sourceId': 

In [62]:
for i in relationships:
    print(f'uploading: {i["$relationshipId"]}')
    service_client.upsert_relationship(
            i["$sourceId"],
            i["$relationshipId"],
            i
        )

uploading: tank74c0258f-2af8-4a62-95ba-f7a9803e47bdconnectsTopipe0791abba-7fc1-49b1-ba25-126d133db4af
uploading: pipe0791abba-7fc1-49b1-ba25-126d133db4afconnectsTopiped83024e9-776c-444e-8f25-04d83c7b227c
uploading: piped83024e9-776c-444e-8f25-04d83c7b227cconnectsTopipe0e49be47-fef2-45a9-b9f7-a0364ec9365d
uploading: pipe0e49be47-fef2-45a9-b9f7-a0364ec9365dconnectsTotankdcd691eb-b335-420d-87f3-d5c3bb3fe2d4
